In [1]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.datasets import cifar100
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
num_classes = 100
batch_size = 128
epochs = 15

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train_onehot = to_categorical(y_train, num_classes)
y_test_onehot = to_categorical(y_test, num_classes)

# datagen = ImageDataGenerator(rescale=1./255)
datagen = ImageDataGenerator()
generator_train = datagen.flow(x_train, y_train_onehot, batch_size=batch_size, seed=0)
generator_test = datagen.flow(x_test, y_test_onehot, batch_size=batch_size, seed=0)

In [3]:
def _build_model():
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',input_shape=x_train[0].shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

#     model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    return model

In [ ]:
def get_model(retrain=True, save=True):
    if retrain:
        model = _build_model()
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit_generator(generator=generator_train,
                                            steps_per_epoch=x_train.shape[0] // batch_size,
                                            epochs=epochs,
                                            validation_data=generator_test,
                                            validation_steps=x_test.shape[0] // batch_size
                                     )
        if save:
            model.save('models/VGG_CIFAR100.h5')                                     
    else:
        model = load_model('models/VGG_CIFAR100.h5')
    return model

model = get_model()
# model = get_model(save=False)

Epoch 1/15
390/390 [==============================] - 26s 67ms/step - loss: 3.7533 - acc: 0.1221 - val_loss: 3.9943 - val_acc: 0.1244
Epoch 2/15
390/390 [==============================] - 22s 57ms/step - loss: 2.9575 - acc: 0.2501 - val_loss: 3.2877 - val_acc: 0.2430
Epoch 3/15
390/390 [==============================] - 22s 56ms/step - loss: 2.4517 - acc: 0.3515 - val_loss: 3.0113 - val_acc: 0.2939
Epoch 4/15
390/390 [==============================] - 22s 56ms/step - loss: 2.0907 - acc: 0.4283 - val_loss: 2.7317 - val_acc: 0.3449
Epoch 5/15
390/390 [==============================] - 22s 56ms/step - loss: 1.8108 - acc: 0.4935 - val_loss: 2.4050 - val_acc: 0.3983
Epoch 6/15
390/390 [==============================] - 22s 55ms/step - loss: 1.5630 - acc: 0.5550 - val_loss: 2.1480 - val_acc: 0.4580
Epoch 7/15
390/390 [==============================] - 21s 55ms/step - loss: 1.3384 - acc: 0.6105 - val_loss: 2.1914 - val_acc: 0.4521
Epoch 8/15
 17/390 [>.............................] - ETA: 19s

In [ ]:
def acc(top_k, y_true):
    correct = 0
    for i in range(y_true.shape[0]):
        if y_true[i] in top_k[i]:
            correct += 1
    return correct/float(y_true.shape[0])

In [ ]:
y_pred = model.predict(x_test)
print("Accuracy: %f" % (np.sum(np.argmax(y_pred, axis=1) == y_test.flatten()) / float(len(y_test))))

k = 5
y_pred_top_k = y_pred.argsort()[:,-k:]
print("Top %d accuracy: %f" % (k, acc(y_pred_top_k, y_test.flatten())))

In [ ]:
model.summary()

In [ ]:
layer_name = 'max_pooling2d_5'
feat_extractor_model = Model(inputs= model.input, outputs = model.get_layer(layer_name).output)

In [ ]:
train_feat = feat_extractor_model.predict(x_train)
print("train features:", train_feat.shape)
test_feat = feat_extractor_model.predict(x_test)
print("test features:", test_feat.shape)

In [ ]:
np.save("vgg_16/train_x", train_feat)
np.save("vgg_16/train_y", y_train)
np.save("vgg_16/test_x", test_feat)
np.save("vgg_16/test_y", y_test)